In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re  # Para usar expresiones regulares
import os

# Ruta del archivo Excel
excel_path = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\PILLA1.xlsx"

# Crear las columnas para el DataFrame
columnas = ['Nombre del Producto', 'Precio Original', 'Precio Final', 'URL del Producto', 'URL de la Imagen', 'SKU']

# Configuración del WebDriver (en este caso, Chrome) para modo headless
options = Options()
options.headless = True  # Ejecutar el navegador en segundo plano

# Inicializar el WebDriver con ChromeDriver Manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL base para la primera página de productos
base_url = "https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page="

# Función para escribir en el archivo Excel después de extraer un producto
def escribir_en_excel(datos_producto):
    # Verificar si el archivo existe
    if os.path.exists(excel_path):
        # Leer el archivo Excel para agregar la nueva fila desde "Hoja1"
        df_existente = pd.read_excel(excel_path, sheet_name='Hoja1', engine='openpyxl')

        # Añadir los datos del nuevo producto al final
        df_nuevo_producto = pd.DataFrame([datos_producto], columns=columnas)

        # Combinar los datos existentes con los nuevos
        df_completo = pd.concat([df_existente, df_nuevo_producto], ignore_index=True)

        # Escribir los datos completos de nuevo al archivo Excel en "Hoja1"
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            df_completo.to_excel(writer, index=False, sheet_name='Hoja1', header=True)

    else:
        # Si el archivo no existe, crear uno nuevo con los datos del producto en "Hoja1"
        df_nuevo_producto = pd.DataFrame([datos_producto], columns=columnas)
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            df_nuevo_producto.to_excel(writer, index=False, sheet_name='Hoja1', header=True)

# Función para extraer los productos de cada página
def extraer_productos_de_pagina(url):
    # Acceder a la página web
    driver.get(url)

    # Esperar hasta que los productos sean visibles
    try:
        # Esperar hasta que los productos sean cargados (hasta 30 segundos)
        WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.vtex-search-result-3-x-galleryItem")))

        # Hacer scroll hacia abajo para cargar más productos
        for i in range(5):  # Hacemos 5 scrolls hacia abajo para cargar más productos (ajusta según sea necesario)
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(3)  # Espera entre scrolls para cargar más productos
        
        # Extraer los productos después de hacer scroll
        productos = driver.find_elements(By.CSS_SELECTOR, "div.vtex-search-result-3-x-galleryItem")
        if not productos:
            print("No se encontraron productos en esta página.")
            return False  # Si no se encuentran productos, retornar False para terminar
        
        # Extraer el nombre, precio original, precio final, URL, URL de la imagen y SKU de cada producto
        for producto in productos:
            try:
                # Extraer el nombre del producto
                nombre_producto = producto.find_element(By.CSS_SELECTOR, "span.vtex-product-summary-2-x-productBrand").text
                
                # Extraer el precio original (concatenar las tres partes)
                precio_original_parts = producto.find_elements(By.CSS_SELECTOR, "span.vtex-product-price-1-x-listPriceValue.vtex-product-price-1-x-listPriceValue--summary.strike")
                precio_original = ''.join([part.text for part in precio_original_parts])
                
                # Extraer el precio final (concatenar las tres partes)
                precio_final_parts = producto.find_elements(By.CSS_SELECTOR, "span.vtex-product-price-1-x-sellingPrice.vtex-product-price-1-x-sellingPrice--summary")
                precio_final = ''.join([part.text for part in precio_final_parts])
                
                # Extraer la URL del producto
                producto_url_element = producto.find_element(By.CSS_SELECTOR, "a.vtex-product-summary-2-x-clearLink")
                producto_url = "https://www.pillin.com" + producto_url_element.get_attribute("href")
                
                # Extraer la URL de la imagen
                imagen_element = producto.find_element(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-skuSelectorItemImageValue")
                imagen_url = imagen_element.get_attribute("src")
                
                # Extraer la SKU: todos los números y guiones de la URL del producto
                sku_match = re.search(r'(\d+-\d+)', producto_url)
                sku = sku_match.group(1) if sku_match else "SKU no encontrada"
                
                # Crear un diccionario con los datos del producto
                datos_producto = {
                    'Nombre del Producto': nombre_producto,
                    'Precio Original': precio_original,
                    'Precio Final': precio_final,
                    'URL del Producto': producto_url,
                    'URL de la Imagen': imagen_url,
                    'SKU': sku
                }
                
                # Escribir los datos del producto en el archivo Excel
                escribir_en_excel(datos_producto)
            
            except Exception as e:
                print("Error al obtener los datos del producto:", e)
        
        return True  # Retornar True para continuar con la siguiente página

    except Exception as e:
        print("Error al esperar por los productos:", e)
        return False  # Si hubo un error, retornar False

# Función para recorrer todas las páginas
def recorrer_paginas():
    pagina = 1
    while True:
        # Construir la URL de la página actual
        url = base_url + str(pagina)
        print(f"Accediendo a la página {pagina}: {url}")
        
        # Intentar extraer productos de la página actual
        productos_encontrados = extraer_productos_de_pagina(url)
        
        # Si no hay productos, terminamos el proceso
        if not productos_encontrados:
            print("No hay más productos en esta página o no se pudo cargar la página.")
            break
        
        # Esperar un poco entre páginas antes de cargar la siguiente (por ejemplo, 5 segundos)
        time.sleep(5)
        
        # Incrementar la página
        pagina += 1

# Llamar a la función para recorrer las páginas
recorrer_paginas()

# Cerrar el navegador al finalizar
driver.quit()


Accediendo a la página 1: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=1
Accediendo a la página 2: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=2
Accediendo a la página 3: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=3
Accediendo a la página 4: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=4
Accediendo a la página 5: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=5
Accediendo a la página 6: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=6
Accediendo a la página 7: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=7
Accediendo a la página 8: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=8
Accediendo a la página 9: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=9
Accediendo a la página 10: https://www.pillin.com/recien-nacido--00-a-12m-?map=agrupacion&page=10
Accediendo a la página 11: https://www